In [1]:
%pip install requests
%pip install bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: C:\Users\pnmat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: C:\Users\pnmat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import requests
import xml.etree.ElementTree as ET

Eseguo la query e ottenere gli ID dei primi 10000 risultati

In [59]:
from Bio import Entrez

Entrez.email = "pnmattia@gmail.com"

In [67]:
def search_articles():
    try:
        query = '"hasplainlanguagesummary"[All Fields] AND fft[Filter]'
        handle = Entrez.esearch(db="pubmed", term=query, retmax=20000)
        results = Entrez.read(handle)
        handle.close()

        return results["IdList"]

    except Exception as e:
        print(f"Error in searching articles: {e}")
        return []

In [58]:
query_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
params = {
    "db": "pubmed",
    "term": "hasplainlanguagesummary",
    "retmax": "10000"
}

response = requests.get(query_url, params=params)
data = ET.fromstring(response.content)
id_list = data.find(".//IdList")
ids = [id_element.text for id_element in id_list.findall(".//Id")]

len(ids)

9999

In [69]:
import urllib.request
import json

def get_preprint_links(ids_list):
    preprint_links_dict = {}  # Dizionario per memorizzare i link correlati per ogni ID

    batch_size = 300
    num_batches = (len(ids_list) + batch_size - 1) // batch_size

    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        batch_ids = ids_list[start_idx:end_idx]

        elinks_json = json.loads(urllib.request.urlopen(f'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?db=pubmed&id={",".join(batch_ids)}&retmode=json&cmd=prlinks').read().decode('utf-8'))

        urls_ = elinks_json['linksets'][0]['idurllist']
        for url_ in urls_:
            preprint_links_dict[url_['id']] = [ x['url']['value'] for x in url_['objurls'] ]
        
    return preprint_links_dict


In [6]:
def get_pub_med_central_links(link_dict):
    return [url for urls in link_dict.values() for url in urls if "pmc" in url]


In [70]:
links = get_pub_med_central_links(get_preprint_links(search_articles()))

In [71]:
len(links)  

4696

In [72]:
import os

def download_pdf(url, folder):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    file_name = os.path.join(folder, url.split("/")[-1])
    with open(file_name, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            f.write(chunk)

In [75]:
from bs4 import BeautifulSoup
from requests.exceptions import RequestException

def bulk_download(urls, folder):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    }

    for url in urls:
        try:
            response = requests.get(url, headers=headers, timeout=5)
            response.raise_for_status()  # Check if the request was successful

            soup = BeautifulSoup(response.content, 'html.parser')

            pdf_link = soup.select_one('a.int-view')
            if pdf_link:
                pdf_url = 'https://www.ncbi.nlm.nih.gov/' + pdf_link['href']
                download_pdf(pdf_url, folder)
                print(f"Downloaded {pdf_url}")
            else:
                print(f"PDF link not found for {url}")
        except RequestException as e:
            print(f"Error accessing {url}: {e}")
            pass


In [76]:
def get_pdfs_url(urls):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    }
    pdf_urls = []
    for url in urls:
        try:
            response = requests.get(url, headers=headers, timeout=5)
            response.raise_for_status()  # Check if the request was successful

            soup = BeautifulSoup(response.content, 'html.parser')

            pdf_link = soup.select_one('a.int-view')
            if pdf_link:
                pdf_url = 'https://www.ncbi.nlm.nih.gov/' + pdf_link['href']
                pdf_urls.append(pdf_url)
            else:
                print(f"PDF link not found for {url}")
        except RequestException as e:
            print(f"Error accessing {url}: {e}")
            pass

In [77]:
pdf_urls = get_pdfs_url(links)

Error accessing https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/37486823/: 403 Client Error: Forbidden for url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10365836/
Error accessing https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/37485492/: 403 Client Error: Forbidden for url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10357067/
Error accessing https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/37485447/: 403 Client Error: Forbidden for url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10357402/
Error accessing https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/37485446/: 403 Client Error: Forbidden for url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10357395/
Error accessing https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/37485445/: 403 Client Error: Forbidden for url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10357398/
Error accessing https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/37485441/: 403 Client Error: Forbidden for url: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10357407/


KeyboardInterrupt: 